In [87]:
#Importação das bibliotecas.

#Para manipulação
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot

#Para Visualização.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import mplcursors

#Para Analises e Modelos.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_graphviz
import graphviz
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pmdarima import auto_arima

#configurações.
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [60]:
df = pd.read_csv('Churn_Modelling.csv', sep=',')
X = df.iloc[:,3:13].values
y = df.iloc[:,13].values

In [61]:
#Mudando dados object.

labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])


ohc = make_column_transformer((OneHotEncoder(categories='auto', sparse=False), [1]), remainder='passthrough')
X = ohc.fit_transform(X)
X = X[:,1:]

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Standart Scaler.
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [95]:
#Criação da rede neural

from keras.models import Sequential
from keras.layers import Dense, Dropout

In [118]:
classifier = Sequential()
#Neuronios ocultos.
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
#Saida
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#Definindo as atualizações no treino.
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier = classifier.fit(X_train, y_train, batch_size=15, epochs=150)




Epoch 1/150
534/534 [==============================] - 1s 974us/step - loss: 0.5164 - accuracy: 0.7936
Epoch 2/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4337 - accuracy: 0.7945
Epoch 3/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4284 - accuracy: 0.7945
Epoch 4/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4247 - accuracy: 0.7945
Epoch 5/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4212 - accuracy: 0.8154
Epoch 6/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4194 - accuracy: 0.8250
Epoch 7/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4173 - accuracy: 0.8275
Epoch 8/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4162 - accuracy: 0.8304
Epoch 9/150
534/534 [==============================] - 1s 1ms/step - loss: 0.4149 - accuracy: 0.8319
Epoch 10/150
534/534 [==============================] - 1s 959us/step - loss: 0.4137 - ac

In [128]:
best_loss = max(classifier.history['accuracy'])
best_loss

0.8608750104904175

In [ ]:
#Previsões.
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
cf_matrix = confusion_matrix(y_pred, y_test)


63/63 [==============================] - 0s 877us/step


In [ ]:
pd.DataFrame(cf_matrix, columns=['True', 'False'], index=['True', 'False'])

,True,False
True,1527,207
False,80,186


In [ ]:
df.head(20)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0
